In [1]:
pip install geopandas matplotlib
import numpy as np
import pandas as pd

In [2]:
inventor_df = pd.read_csv('C:/Users/aarus/Downloads/g_inventor_disambiguated.tsv/g_inventor_disambiguated.tsv',sep='\t')
location_df = pd.read_csv('C:/Users/aarus/Downloads/g_location_disambiguated.tsv/g_location_disambiguated.tsv',sep='\t')

In [3]:
inventor_df.info()

In [4]:
inventor_df=inventor_df.drop(['disambig_inventor_name_first', 'disambig_inventor_name_last', 'inventor_sequence'], axis=1)
inventor_df=inventor_df.sort_values(by=['inventor_id'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21467473 entries, 0 to 21467472
Data columns (total 7 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   patent_id                     object
 1   inventor_sequence             int64 
 2   inventor_id                   object
 3   disambig_inventor_name_first  object
 4   disambig_inventor_name_last   object
 5   gender_code                   object
 6   location_id                   object
dtypes: int64(1), object(6)
memory usage: 1.1+ GB


In [8]:
patent_count= inventor_df.groupby('inventor_id')['patent_id'].count().reset_index()
patent_count.columns = ['inventor_id', 'patent_count']
inventor_df = pd.merge(inventor_df, patent_count, on='inventor_id', how='left')
inventor_df = pd.merge(inventor_df, location_df, on='location_id', how='left')

In [1]:
df_temp= inventor_df.drop('patent_id', axis=1)
df_temp= df_temp.drop_duplicates()
most_common_location = df_temp.groupby('inventor_id')['location_id'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA).reset_index()

# Rename the columns for clarity
most_common_location.columns = ['inventor_id', 'most_common_location_id']

# Merge the most_common_location DataFrame with the original inventor_df DataFrame
df_temp = pd.merge(df_temp, most_common_location, on='inventor_id', how='left')
df_temp = df_temp.drop(['disambig_city', 'disambig_state', 'disambig_country', 'latitude', 'longitude', 'county', 'state_fips', 'county_fips'], axis=1)
df_temp = df_temp.merge(location_df, left_on='most_common_location_id', right_on='location_id', how='left')
df_temp= df_temp.drop_duplicates(subset=['inventor_id'])

NameError: name 'inventor_df' is not defined

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Create a GeoDataFrame from inventor_df
gdf = gpd.GeoDataFrame(df_temp, 
                       geometry=gpd.points_from_xy(df_temp.longitude, df_temp.latitude))

# World basemap
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the world map
fig, ax = plt.subplots(figsize=(15, 10))
world.plot(ax=ax, color='lightgray')

# Plot inventor locations on top of the world map, color intensity based on patent_count
gdf.plot(ax=ax, marker='o', alpha=0.7, cmap='viridis', markersize=10, norm=LogNorm(), c='patent_count', legend=True)

# Add colorbar
cbar = plt.colorbar(ax.collections[0], label='Patent Count', orientation='vertical')

# Remove inventor_id labels
ax.set_title('Inventor Locations on World Map')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
location_counts = df_temp.groupby(['latitude', 'longitude']).size().reset_index(name='inventor_count')

# Create a GeoDataFrame from location_counts
gdf = gpd.GeoDataFrame(location_counts, 
                       geometry=gpd.points_from_xy(location_counts.longitude, location_counts.latitude))

# World basemap
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the world map
fig, ax = plt.subplots(figsize=(15, 10))
world.plot(ax=ax, color='lightgray')

# Plot inventor locations on top of the world map, dot size based on inventor_count
gdf.plot(ax=ax, marker='o', alpha=0.7, cmap='viridis', markersize=gdf['inventor_count']*2, norm=LogNorm(), legend=True)

# Add colorbar
cbar = plt.colorbar(ax.collections[0], label='Inventor Count', orientation='vertical')

# Remove inventor_id labels
ax.set_title('Inventor Locations on World Map (Dot Size based on Inventor Count)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()


In [ ]:
df_temp2 = df_temp[df_temp['disambig_country'] == 'US']

from mpl_toolkits.axes_grid1 import make_axes_locatable
# Load the US states GeoDataFrame
us_states = gpd.read_file('C:/Users/aarus/Downloads/cb_2021_us_state_20m/cb_2021_us_state_20m.shp')

# Merge inventor_df with us_states based on the common geometry
gdf = gpd.GeoDataFrame(df_temp2, 
                       geometry=gpd.points_from_xy(df_temp2.longitude, df_temp2.latitude))

# Spatial join to associate inventors with US states
gdf_us_states = gpd.sjoin(gdf, us_states, how='left', op='within')

# Aggregate by state, counting the number of inventors in each state
state_counts = gdf_us_states.groupby('STUSPS').size().reset_index(name='inventor_count')

# Merge state_counts with us_states based on state code (STUSPS)
us_states = us_states.merge(state_counts, left_on='STUSPS', right_on='STUSPS', how='left')

# Plot the choropleth map
fig, ax = plt.subplots(figsize=(25, 20))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

us_states.plot(column='inventor_count', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, cax=cax)

# Add colorbar
cax.set_title('Inventor Count', pad=10)

# Remove x and y axis labels
ax.set_xticks([])
ax.set_yticks([])

# Set a title
ax.set_title('Inventors in US States')

plt.show()
